Code to Calculate Mie Theory for Core-Shell spheres using theory from "Improved recursive algorithm for light scattering by a multilayered sphere" from Applied Optics Yang 2003

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
from pylab import *
import scipy.constants as sc
import scipy as scipy
from scipy import optimize
import matplotlib
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
import matplotlib.ticker as ticker
%matplotlib inline

In [45]:
import cmath
import pandas as pd
import scipy.constants as sc
import scipy.special
import scipy
import numpy as np
import math
import string as string
import csv

inputdata = "/home/bldrdge1/Au_Si.txt" #File containing refractive indices of material

data1 = pd.read_csv(inputdata, sep='\t', header=None, skiprows=1).values 
lamda = data1[:,0] #wavelengths in nm

# column1 = int(input("Column contaning real refrac index of outermost shell:\t"))
# column2 = int(input("Column contaning imaginary refrac index of outermost shell:\t"))
# column3 = int(input("Column contaning real refrac index of middle shell:\t"))
# column4 = int(input("Column contaning imaginary refrac index of middle shell:\t"))
# column5 = int(input("Column contaning real refrac index of inner core:\t"))
# column6 = int(input("Column contaning imaginary refrac index of inner core:\t"))

column1 = 1
column2 = 2
column3 = 3
column4 = 4
column5 = 1
column6 = 2

n3 = data1[:,column1] #real part of the refractive index of outermost shell
k3 = data1[:,column2] #imaginary part of the refractive index of outermost shell
N3 = n3+(k3*1j) #complex refractive index of outermost shell

n2 = data1[:,column3] #real part of the refractive index of middle shell
k2 = data1[:,column4] #imaginary part of the refractive index of middle shell
N2 = n2+(k2*1j) #complex refractive index of middle shell

n1 = data1[:,column5] #real part of the refractive index of inner core
k1 = data1[:,column6] #imaginary part of the refractive index of inner core
N1 = n1+(k1*1j) #complex refractive index of inner sphere

n_m = 1.33 #real part of the refractive index of medium
k_m = 0 #imaginary part of refractive index of medium
N_m = n_m+(k_m*1j)

m_m = 1 #relative refractive index of medium
m3 = N3/N_m
m2 = N2/N_m
m1 = N1/N_m

#######unsure importance
linemax = 702
#######

output = "\[Lambda] QEXT QNF-OUT QNF-INT QNF-IN QSCA QABS CEXT CSCA CABS"

# r3 = float(input("Outer radius of the outermost shell:\t")) #outer radius of outer shell
# r2 = float(input("Outer radius of the middle shell:\t")) #outer radius of middle shell
# r1 = float(input("Radius of the inner core sphere:\t")) #radius of inner sphere

r3 = 28
r2 = 26
r1 = 25

def x1(r): return (2*sc.pi*r*N_m)/lamda
def x2(r): return (2*sc.pi*r*N_m)/lamda
def x3(r): return (2*sc.pi*r*N_m)/lamda
def y1(r): return (m1*2*sc.pi*r*N_m)/lamda
def y2(r): return (m2*2*sc.pi*r*N_m)/lamda
def y3(r): return (m3*2*sc.pi*r*N_m)/lamda
def z1(r): return (m2*2*sc.pi*r*N_m)/lamda
def z2(r): return (m3*2*sc.pi*r*N_m)/lamda
def z3(r): return (m_m*2*sc.pi*r*N_m)/lamda

largest = max(max(x1(r1).real), 
              max(x2(r2).real), 
              max(x3(r3).real), 
              max(y1(r1).real), 
              max(y2(r2).real), 
              max(y3(r3).real), 
              max(z1(r1).real), 
              max(z2(r2).real), 
              max(z3(r3).real))
lastTerm = math.ceil(abs(largest+4.05*(largest**(1/3))+2))
#The previous two variables were chosen from Eq 30 from Yang 2003

# def phi(n,z): return z*scipy.special.spherical_jn(n,z) #Riccati-Bessel Function where n is the order and z is the parameter
# def Dphi(n,z): return scipy.special.spherical_jn(n,z, True) #Derivative of Bessel Fxn
# def chi(n,z): return -z*scipy.special.spherical_jn(n,z) #Negative Bessel Fxn
# def Dchi(n,z): return -scipy.special.spherical_jn(n,z) #Derivative of negative Bessel Fxn
# def zeta(n,z): return z*(scipy.special.spherical_jn(n,z)+(1j*scipy.special.spherical_yn(n,z)))
# def Dzeta(n,z): return derivative
# def D1(n,z): return Dphi(n,z)/phi(n,z)
# def D3(n,z): return Dzeta(n,z)/zeta(n,z)
# def RR(n,z): return phi(n,z)/zeta(n,z)

def phi(n,z): return z*scipy.special.spherical_jn(n,z) #Spherical Bessel Function where n is the order and z is the parameter
def Dphi(n,z): return (z*scipy.special.spherical_jn(n,z, True))+(scipy.special.spherical_jn(n,z)) #Derivative of phi using product rule
def chi(n,z): return -z*scipy.special.spherical_jn(n,z) #Negative Bessel Fxn
def Dchi(n,z): return -(z*scipy.special.spherical_jn(n,z, True))-(scipy.special.spherical_jn(n,z)) #Derivative of negative phi using product rule
def zeta(n,z): return z*(scipy.special.spherical_jn(n,z)+(1j*scipy.special.spherical_yn(n,z))) #Spherical Hankel function (not in a python library, had to find alternate definition https://en.wikipedia.org/wiki/Bessel_function#Hankel_functions:_H(1)α,_H(2)α)
def Dzeta(n,z): return (scipy.special.spherical_jn(n,z)+(1j*scipy.special.spherical_yn(n,z)))+(((n*(scipy.special.spherical_jn(n,z)+(1j*scipy.special.spherical_yn(n,z))))/z)-(scipy.special.spherical_jn(n+1,z)+(1j*scipy.special.spherical_yn(n+1,z)))) #Derivative of zeta according to https://mathworld.wolfram.com/HankelFunctionoftheFirstKind.html
def D1(n,z): return Dphi(n,z)/phi(n,z)
def D3(n,z): return Dzeta(n,z)/zeta(n,z)
def RR(n,z): return phi(n,z)/zeta(n,z)

xTable = np.vstack([np.round([phi(n,x1(r1)), Dphi(n,x1(r1)), 
                    zeta(n,x1(r1)), Dzeta(n,x1(r1)), 
                    phi(n,x2(r2)), Dphi(n,x2(r2)), 
                    zeta(n,x2(r2)), Dzeta(n,x2(r2))],6) for n in np.arange(1,6)])

yTable = np.vstack([np.round([phi(n,y1(r1)), Dphi(n,y1(r1)), 
                    zeta(n,y1(r1)), Dzeta(n,y1(r1)), 
                    phi(n,y2(r2)), Dphi(n,y2(r2)), 
                    zeta(n,y2(r2)), Dzeta(n,y2(r2))],6) for n in np.arange(1,6)])

zTable = np.vstack([np.round([phi(n,z1(r1)), Dphi(n,z1(r1)), 
                    zeta(n,z1(r1)), Dzeta(n,z1(r1)), 
                    phi(n,z2(r2)), Dphi(n,z2(r2)), 
                    zeta(n,z2(r2)), Dzeta(n,z2(r2))],6) for n in np.arange(1,6)])

d1Table = np.vstack([np.round([D1(n,y1(r1)), D1(n,y2(r2)), 
                    D1(n,z1(r1)), D1(n,z2(r2))],6) for n in np.arange(1,6)])

d3Table = np.vstack([np.round([D3(n,y1(r1)), D3(n,y2(r2)), 
                    D3(n,z1(r1)), D3(n,z2(r2))],6) for n in np.arange(1,6)])

rrTable = np.vstack([np.round([RR(n,y1(r1)), RR(n,y2(r2)), 
                    RR(n,z1(r1)), RR(n,z2(r2))],6) for n in np.arange(1,6)])

def A2(n,r): return (RR(n,z1(r)))*((m2*D1(n,y1(r))-m1*D1(n,z1(r)))/(m2*D1(n,y1(r))-m1*D3(n,z1(r))))
def Ha2(n,r_1,r_2): return ((RR(n,y2(r_2))*D1(n,y2(r_2)))-(A2(n,r_1)*D3(n,y2(r_2))))/(RR(n,y2(r_2))-A2(n,r_1))
def A3(n, r_1, r_2): return (RR(n,z2(r_2)))*(((m3*Ha2(n,r_1,r_2))-(m2*D1(n,z2(r_2))))/((m3*Ha2(n,r_1,r_2))-(m2*D3(n,z2(r_2)))))
def Ha3(n,r_1,r_2,r_3): return ((RR(n,y3(r_3))*D1(n,y3(r_3)))-(A3(n,r_1,r_2)*D3(n,y3(r_3))))/(RR(n,y3(r_3))-A3(n,r_1,r_2))
def B2(n,r_1): return RR(n,z1(r_1))*(((m1*D1(n,y1(r_1)))-(m2*D1(n,z1(r_1))))/((m1*D1(n,y1(r_1)))-(m2*D3(n,z1(r_1)))))
def Hb2(n,r_1,r_2): return ((RR(n,y2(r_2))*D1(n,y2(r_2)))-(B2(n,r_1)*D3(n,y2(r_2))))/(RR(n,y2(r_2))-B2(n,r_1))
def B3(n,r_1,r_2): return RR(n,z2(r_2))*(((m2*Hb2(n,r_1,r_2))-(m3*D1(n,z2(r_2))))/((m2*Hb2(n,r_1,r_2))-(m3*D3(n,z2(r_2)))))
def Hb3(n,r_1,r_2,r_3): return ((RR(n,y3(r_3))*D1(n,y3(r_3)))-(B3(n,r_1,r_2)*D3(n,y3(r_3))))/(RR(n,y3(r_3)-B3(n,r_1,r_2)))
def Am(n,r_1,r_2,r_3): return RR(n,z3(r_3))*(((m_m*Ha3(n,r_1,r_2,r_3))-(m3*D1(n,z3(r_3))))/((m_m*Ha3(n,r_1,r_2,r_3))-(m3*D3(n,z3(r_3)))))
def Bm(n,r_1,r_2,r_3): return RR(n,z3(r_3))*(((m3*Hb3(n,r_1,r_2,r_3))-(m_m*D1(n,z3(r_3))))/((m3*Hb3(n,r_1,r_2,r_3))-(m_m*D3(n,z3(r_3)))))

a2b2Table = np.vstack([np.round([A2(n,r1), Ha2(n,r1,r2), 
                        B2(n,r1), Hb2(n,r1,r2)],6) for n in np.arange(1,6)])
                                             
ambmTable = np.vstack([np.round([Am(n,r1,r2,r3), Bm(n,r1,r2,r3)],6) for n in np.arange(1,6)])

#The following code is inspired by Absorption and Scattering by Small Particles p.103 Bohren & Huffman

kappa = (2*sc.pi*N_m)/lamda
def vext(r_1,r_2,r_3): 
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+((2*n)+1)*(Am(n,r_1,r_2,r_3).real+Bm(n,r_1,r_2,r_3).real)
    return (2*sc.pi*sumd)/kappa**2

def vsca(r_1,r_2,r_3):
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+((2*n)+1)*((np.abs(Am(n,r_1,r_2,r_3))**2)+(np.abs(Bm(n,r_1,r_2,r_3))**2))
    return (2*sc.pi*sumd)/kappa**2

def vabs(r_1,r_2,r_3): return vext(r_1,r_2,r_3)-vsca(r_1,r_2,r_3)

def Qext(r_1,r_2,r_3):
    if (m1==m2).all() and (m2==m3).all() and (m3==m_m).all():
        return 0
    elif (m2==m3).all() and (m3==m_m).all():
        return (vext(r_1,r_2,r_3))/(sc.pi*(r_1**2))
    elif (m3==m_m).all():
        return (vext(r_1,r_2,r_3))/(sc.pi*(r_2**2))
    elif (m3!=m_m).all():
        return (vext(r_1,r_2,r_3))/(sc.pi*(r_3**2))
    else:
        return "Error"

def Qsca(r_1,r_2,r_3):
    if (m1==m2).all() and (m2==m3).all() and (m3==m_m).all():
        return 0
    elif (m2==m3).all() and (m3==m_m).all():
        return (vsca(r_1,r_2,r_3))/(sc.pi*(r_1**2))
    elif (m3==m_m).all():
        return (vsca(r_1,r_2,r_3))/(sc.pi*(r_2**2))
    elif (m3!=m_m).all():
        return (vsca(r_1,r_2,r_3))/(sc.pi*(r_3**2))
    else:
        return "Error"
    
def Qabs(r_1,r_2,r_3): return Qext(r_1,r_2,r_3)-Qsca(r_1,r_2,r_3)

def SphericalHankelH2(n,z):
    return scipy.special.spherical_jn(n,z)-(1j*scipy.special.spherical_yn(n,z)) #from https://en.wikipedia.org/wiki/Bessel_function#Hankel_functions:_H(1)α,_H(2)α

def Qnf_out(r_1,r_2,r_3):
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+(((np.abs(Am(n,r_1,r_2,r_3)))**2)*((n+1)*(np.abs(SphericalHankelH2(n-1,z3(r_3))))**2)+(n*(np.abs(SphericalHankelH2(n+1,z3(r_3))))**2))+(((2*n)+1)*((np.abs(Bm(n,r_1,r_2,r_3)))**2)*(np.abs(SphericalHankelH2(n,z3(r_3))))**2)
    return 0.5*sumd

def Qnf_int(r_1,r_2):
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+(((np.abs(A3(n,r_1,r_2)))**2)*((n+1)*(np.abs(SphericalHankelH2(n-1,z2(r_2))))**2)+(n*(np.abs(SphericalHankelH2(n+1,z2(r_2))))**2))+(((2*n)+1)*((np.abs(B3(n,r_1,r_2)))**2)*(np.abs(SphericalHankelH2(n,z2(r_2))))**2)
    return 0.5*sumd

def Qnf_in(r_1):
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+(((np.abs(A2(n,r_1)))**2)*((n+1)*(np.abs(SphericalHankelH2(n-1,z1(r_1))))**2)+(n*(np.abs(SphericalHankelH2(n+1,z1(r_1))))**2))+(((2*n)+1)*((np.abs(B2(n,r_1)))**2)*(np.abs(SphericalHankelH2(n,z1(r_1))))**2)
    return 0.5*sumd

##################

outputFile = 'mie_EXTABSNF.txt'

file = open(outputFile, "w+")

file.write("\t".join(["Lambda","Qext","Qnf_out","Qnf_int","Qnf_in","Qsca","Qabs","vext","vsca","vabs"]))
file.write("\n")

import time

start_time = time.time()
    
list1 = lamda.real
list2 = Qext(r1,r2,r3).real
list3 = Qnf_out(r1,r2,r3).real
list4 = Qnf_int(r1,r2).real
list5 = Qnf_in(r1).real
list6 = Qsca(r1,r2,r3).real
list7 = Qabs(r1,r2,r3).real
list8 = vext(r1,r2,r3).real
list9 = vsca(r1,r2,r3).real
list10 = vabs(r1,r2,r3).real


for i in np.arange(1,len(lamda)):
    text = "\t".join([str(list1[i]),
                     str(list2[i]),
                     str(list3[i]),
                     str(list4[i]),
                     str(list5[i]),
                     str(list6[i]),
                     str(list7[i]),
                     str(list8[i]),
                     str(list9[i]),
                     str(list10[i])])
    file.write(text+"\n")
    
file.close()

print("\n\nMy program took", time.time() - start_time,"seconds to run")



My program took 40.99231791496277 seconds to run
